# Concepts of Agentic AI

* agentic ai behaves with agency and autonomy, can problem-solve and react to different real-life situation
* flipped interaction pattern -> agentic system interacts with humans and other systems, without us necessary telling it what to do, in order to accomplish a goal we gave it

* agent loop (general)
    * human sets a goal
    * ai takes actions towards the goal using available resources (APIs, SQLs, etc)
        * prompt -> response -> action
        * get feedback on result from ai
    * termination criteria reached, return result

* agent loop
    * construct prompt
    * generate response
    * parse response
    * execute action
    * convert results to string
    * continue?

In [1]:
import os
with open("openai_token", "r") as file:
    openai_token = file.read().strip()
os.environ["OPENAI_API_KEY"] = openai_token

In [13]:
# scaffolding example
import base64
from typing import List, Dict
from litellm import completion

def generate_response(messages:List[Dict]) -> str:
    response = completion(
        model="openai/gpt-4o",
        messages=messages,
        max_tokens=1024,
        #temperature=0.7
    )
    return response.choices[0].message.content

messages = [
    {"role": "system", "content": "You are an expert software engineer that prefers functional programming. Return Base64 encoded string only"},
    {"role": "user", "content": "Write a function to swap the keys and values in a dictionary."}
]

response = generate_response(messages)
print(response)

ZGVmIHN3YXBfZGljdChkKTogDQogICAgcmV0dXJuIHt2OiBrIGZvciBrLCB2IGluIGQuaXRlbXMoKX0=


In [15]:
# lets try a bit of programmatic prompting
import json

code_spec = {
    "name": "swap_keys_values",
    "description": "A function that swaps the keys and values in a dictionary.",
    "parameters": {
        "d": "A dictionary with unique values."}
    }

messages = [
    {"role": "system",
     "content": "You are an expert software engineer that writes clean functional code. You always document your functions."},
    {"role": "user", "content": f"Please implement: {json.dumps(code_spec)}"}
]

response = generate_response(messages)
print(response)

```python
def swap_keys_values(d):
    """
    Swaps the keys and values in a dictionary where all values are unique.
    
    Parameters:
    d (dict): A dictionary with unique values.
    
    Returns:
    dict: A new dictionary with keys and values swapped. The original values become keys,
          and the original keys become values.
    
    Example:
    >>> swap_keys_values({'a': 1, 'b': 2, 'c': 3})
    {1: 'a', 2: 'b', 3: 'c'}
    """
    return {value: key for key, value in d.items()}
```

This function creates a new dictionary by iterating over the items of the input dictionary and constructs key-value pairs by reversing the original key-value relationship. Since the original values are required to be unique, they can safely be used as keys in the new dictionary.


In [16]:
# models do not recall previous conversations, so they need to be supplied in the messaging part
messages = [
   {"role": "system", "content": "You are an expert software engineer that prefers functional programming."},
   {"role": "user", "content": "Write a function to swap the keys and values in a dictionary."},
   
   # Here is the assistant's response from the previous step
   # with the code. This gives it "memory" of the previous
   # interaction.
   {"role": "assistant", "content": response},
   {"role": "user", "content": "Update the function to include documentation."}
]

response = generate_response(messages)
print(response)

Certainly! The function already contains a docstring for documentation, but let me expand it a little further to make it more comprehensive, detailing more about the assumptions and functionality:

```python
def swap_keys_values(d):
    """
    Swaps the keys and values in a dictionary, assuming all values are unique.
    
    Parameters:
    d (dict): A dictionary where all the values are unique and hashable. 
    
    Returns:
    dict: A new dictionary with the keys and values swapped. The values of the input
          dictionary become the keys, and the keys become the corresponding values.
          
    Raises:
    ValueError: If any duplicates are found among the values in the input dictionary,
                as they would cause collisions in the new dictionary where those values
                become keys.

    Example:
    >>> swap_keys_values({'a': 1, 'b': 2, 'c': 3})
    {1: 'a', 2: 'b', 3: 'c'}

    Note:
    - This function assumes that all values in the input dictionary

* system -> setting up the agent, most important part
* user -> user inputs
* assistant -> agent outputs  
**-> memory through user + assistant history**

In [47]:
import json
from typing import List, Dict
from litellm import completion


def generate_response(messages:List[Dict]) -> str:
    response = completion(
        model="openai/gpt-4o",
        messages=messages,
        max_tokens=1024,
        #temperature=0.7
    )
    return response.choices[0].message.content

# first prompt
messages = [
   {"role": "system",
    "content":
        """You are an expert software engineer that prefers functional programming.
        You return parsable json only, with explanations under key exp and code under key code."""},
   {"role": "user",
    "content":
        "Write a function to swap the keys and values in a dictionary."}]

response = generate_response(messages)
print(response)

{
  "exp": "The function `swap_keys_values` takes a dictionary as input and returns a new dictionary where the keys and values have been swapped. It assumes that the values in the input dictionary are unique and hashable, as they will be used as keys in the new dictionary. The implementation is done in a functional style using a dictionary comprehension.",
  "code": "def swap_keys_values(d):\n    return {v: k for k, v in d.items()}"
}


In [48]:
messages.append({"role": "assistant", "content": response})
messages.append({"role": "user", "content":
        """Update the function to include documentation, parameter descriptions,
        return value description, example usage, edge cases."""})

response = generate_response(messages)
print(json.loads(response)["code"])

def swap_keys_values(d):
    """
    Swap the keys and values in a given dictionary.
    
    Parameters:
    d (dict): The dictionary to swap keys and values from. Assumes all values are unique and hashable.
    
    Returns:
    dict: A new dictionary with keys and values swapped.
    
    Example:
    >>> swap_keys_values({'a': 1, 'b': 2, 'c': 3})
    {1: 'a', 2: 'b', 3: 'c'}

    Edge Cases:
    - If the input dictionary is empty, the function returns an empty dictionary.
    - If the dictionary has non-hashable values (e.g., lists), the function will raise an error.
    - If multiple keys map to the same value in the input dictionary (not allowed), a `ValueError` will occur.
    """
    return {v: k for k, v in d.items()}

# Example usage
# swapped_dict = swap_keys_values({'a': 1, 'b': 2, 'c': 3})
# print(swapped_dict)  # Output: {1: 'a', 2: 'b', 3: 'c'}


In [50]:
messages.append({"role": "assistant", "content": response})
messages.append({"role": "user", "content":
       """Add unit test for the function using unittest framework,
       test should cover typical usage, edge cases, error cases and various
       input types. The results should include original function and the tests."""})

response = generate_response(messages)
print(json.loads(response)["code"])

def swap_keys_values(d):
    """
    Swap the keys and values in a given dictionary.
    
    Parameters:
    d (dict): The dictionary to swap keys and values from. Assumes all values are unique and hashable.
    
    Returns:
    dict: A new dictionary with keys and values swapped.
    
    Example:
    >>> swap_keys_values({'a': 1, 'b': 2, 'c': 3})
    {1: 'a', 2: 'b', 3: 'c'}

    Edge Cases:
    - If the input dictionary is empty, the function returns an empty dictionary.
    - If the dictionary has non-hashable values (e.g., lists), the function will raise an error.
    - If multiple keys map to the same value in the input dictionary (not allowed), a `ValueError` will occur.
    """
    return {v: k for k, v in d.items()}

import unittest

class TestSwapKeysValues(unittest.TestCase):
    def test_typical_usage(self):
        self.assertEqual(swap_keys_values({'a': 1, 'b': 2, 'c': 3}), {1: 'a', 2: 'b', 3: 'c'})

    def test_empty_dict(self):
        self.assertEqual(swap_keys_valu